In [227]:
import re
from json import dump
import requests

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 51) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)


for page in N_PAGES:
    url = BASE_URL + f"/rent/?bedrooms=1-any&state=vic&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])





In [213]:
len(url_links)

2001

In [229]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    if(bs_object.find("h1", {"class": "css-164r41r"})is None):
        continue;
    else:
        property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    information = bs_object.find("div", {"data-testid": "strip-content-list"})
    if information :
        property_metadata[property_url]['external_information'] = information.text
    else: 
        property_metadata[property_url]['external_information'] = None

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example.json', 'w') as f:
    dump(property_metadata, f)

In [231]:
# begin code
url_links = []

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?bedrooms=1-any&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])




In [ ]:
property_metadata = defaultdict(dict)
#  for each url, scrape some basic metadata
for property_url in url_links:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    if(bs_object.find("h1", {"class": "css-164r41r"})is None):
        continue;
    else:
        property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    information = bs_object.find("div", {"data-testid": "strip-content-list"})
    if information :
        property_metadata[property_url]['external_information'] = information.text
    else: 
        property_metadata[property_url]['external_information'] = None

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example2.json', 'w') as f:
    dump(property_metadata, f)

In [269]:
# begin code
url_links = []

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/?ptype=house&state=vic&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

In [313]:
property_metadata = defaultdict(dict)
#  for each url, scrape some basic metadata
for property_url in url_links:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    if(bs_object.find("h1", {"class": "css-164r41r"})is None):
        continue;
    else:
        property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    
    information = bs_object.find("div", {"data-testid": "strip-content-list"})
    if information :
        property_metadata[property_url]['external_information'] = information.text
    else: 
        property_metadata[property_url]['external_information'] = None

    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example3.json', 'w') as f:
    dump(property_metadata, f)

In [241]:
import json
import pandas as pd
  
# Opening JSON file
f = open('../data/raw/example.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
df2 = pd.DataFrame.from_dict(data, orient="index")

In [242]:
import json
import pandas as pd
  
# Opening JSON file
f2 = open('../data/raw/example2.json')
  
data_2 = json.load(f2)
df4 = pd.DataFrame.from_dict(data_2, orient="index")
df4

,name,cost_text,coordinates,rooms,external_information,desc
https://www.domain.com.au/unit-6-125-ferguson-street-williamstown-vic-3016-16022370?topspot=1,Unit 6/125 Ferguson Street Williamstown VIC 3016,$310,"[-37.8586754, 144.8933017]","[[1 Bed], [1 Bath], [1 Parking]]","Available from Friday, 2nd September 2022Bond ...",Kristen Standish
https://www.domain.com.au/46-princes-st-carlton-north-vic-3054-16067211,46 Princes St Carlton North VIC 3054,$550 weekly,"[-37.7926524, 144.9740802]","[[2 Beds], [1 Bath], []]","Available from Thursday, 15th September 2022Bo...",Rebecca Pace
https://www.domain.com.au/1652-dandenong-road-oakleigh-east-vic-3166-16067192,1652 Dandenong Road Oakleigh East VIC 3166,$475,"[-37.9070769, 145.1149659]","[[2 Beds], [2 Baths], [1 Parking]]","Available from Monday, 5th September 2022Bond ...",Christian Hidayat
https://www.domain.com.au/26-55-alexandra-st-st-kilda-east-vic-3183-16067159,26/55 Alexandra St St Kilda East VIC 3183,$330 weekly,"[-37.8644943, 145.0029153]","[[1 Bed], [1 Bath], [1 Parking]]","Available from Thursday, 1st September 2022Bon...",Leasing Concierge
https://www.domain.com.au/3405-288-spencer-street-melbourne-vic-3000-16067117,3405/288 Spencer Street Melbourne VIC 3000,$440,"[-37.813775, 144.9520948]","[[1 Bed], [1 Bath], []]","Available from Friday, 16th September 2022Bond...",Kim Te
...,...,...,...,...,...,...
https://www.domain.com.au/4-3-fulton-street-st-kilda-east-vic-3183-16022094,4/3 Fulton Street St Kilda East VIC 3183,$525 pw,"[-37.861077, 144.9974369]","[[2 Beds], [1 Bath], [1 Parking]]",Date Available: Available NowBond $2281,Artem Chereshkov
https://www.domain.com.au/4-36-blessington-street-st-kilda-vic-3182-16022078,4/36 Blessington Street St Kilda VIC 3182,$300 per week,"[-37.8697373, 144.9819009]","[[1 Bed], [1 Bath], [1 Parking]]",Date Available: Available NowBond $1303,Xavier George
https://www.domain.com.au/109-easey-street-collingwood-vic-3066-16021861,109 Easey Street Collingwood VIC 3066,$750,"[-37.7983554, 144.9907584]","[[3 Beds], [1 Bath], [1 Parking]]",Date Available: Available NowBond $3259,Nigel Chee
https://www.domain.com.au/4-palfreyman-street-south-yarra-vic-3141-16021762,4 Palfreyman Street South Yarra VIC 3141,$980 Per Week,"[-37.844978, 144.997811]","[[3 Beds], [1 Bath], [1 Parking]]",Date Available: Available NowBond $4900,Tom Koller


In [314]:
import json
import pandas as pd
  
# Opening JSON file
f2 = open('../data/raw/example3.json')
  
data_3 = json.load(f2)
df5 = pd.DataFrame.from_dict(data_3, orient="index")
df5

,name,cost_text,coordinates,rooms,external_information,desc
https://www.domain.com.au/12-macfarlan-street-south-yarra-vic-3141-16044182?topspot=1,12 MacFarlan Street South Yarra VIC 3141,$625 per week pw,"[-37.840127, 144.988448]","[[2 Beds], [1 Bath], []]",Date Available: Available NowBond $2709,Joshua Ling
https://www.domain.com.au/98-johnston-st-collingwood-vic-3066-16067212,98 Johnston St Collingwood VIC 3066,"$1,150 weekly","[-37.799194, 144.98827]","[[3 Beds], [1 Bath], []]","Available from Wednesday, 7th September 2022Bo...",Lauren Allingham
https://www.domain.com.au/46-princes-st-carlton-north-vic-3054-16067211,46 Princes St Carlton North VIC 3054,$550 weekly,"[-37.7926524, 144.9740802]","[[2 Beds], [1 Bath], []]","Available from Thursday, 15th September 2022Bo...",Rebecca Pace
https://www.domain.com.au/26-victoria-grove-hawthorn-east-vic-3123-16067087,26 Victoria Grove Hawthorn East VIC 3123,$995.00,"[-37.8220902, 145.0488853]","[[3 Beds], [2 Baths], [2 Parking]]",Date Available: Available NowBond $6485,Tom Macrae
https://www.domain.com.au/63-mcarthur-street-malvern-vic-3144-16067027,63 McArthur Street Malvern VIC 3144,$580.00,"[-37.8535983, 145.0395781]","[[2 Beds], [1 Bath], [2 Parking]]","Available from Friday, 16th September 2022Bond...",Tom Macrae
...,...,...,...,...,...,...
https://www.domain.com.au/7-atherton-way-werribee-vic-3030-16020564,7 Atherton Way Werribee VIC 3030,$410 pw,"[-37.9010132, 144.598294]","[[4 Beds], [2 Baths], [2 Parking]]",Date Available: Available NowBond $1782,Centrally located in the heart of the popular ...
https://www.domain.com.au/27-gordon-street-toorak-vic-3142-16020462,27 Gordon Street Toorak VIC 3142,$695.00 Per Week,"[-37.8437917, 145.0068204]","[[2 Beds], [1 Bath], [1 Parking]]",Date Available: Available NowBond $3020,This charming light filled two bedroom residen...
https://www.domain.com.au/18-college-road-werribee-vic-3030-16020229,18 College Road Werribee VIC 3030,$390,"[-37.90651829999999, 144.6654842]","[[4 Beds], [2 Baths], [2 Parking]]",Date Available: Available NowBond $1695,"In keeping with its cottage charm, this proper..."
https://www.domain.com.au/b30-berth-martha-cove-marina-safety-beach-vic-3936-16020013,B30 BERTH MARTHA COVE MARINA Safety Beach VIC ...,$160.00,"[-38.3352183, 144.9942305]","[[1 Bed], [1 Bath], []]","Available from Friday, 25th November 2022Bond ...","BERTH 30 MARTHA COVE, SAFETY BEACH - BERTH ONL..."


In [243]:
l = []
for i in range(len(df2)):
    if(len(df2['rooms'][i]) != 0):
        l.append(True)
    else:
        l.append(False)
df2 = df2[l]

In [245]:
l = []
for i in range(len(df4)):
    if(len(df4['rooms'][i]) != 0):
        l.append(True)
    else:
        l.append(False)
df4 = df4[l]


In [315]:
l = []
for i in range(len(df4)):
    if(len(df5['rooms'][i]) != 0):
        l.append(True)
    else:
        l.append(False)
df5 = df5[l]

In [316]:
df3 = pd.concat([df2, df4, df5])

In [317]:
df3 = df3.reset_index()
df3.rename(columns={'index':'link'}, inplace = True)

In [318]:
import re
bedroom_list = []
bathroom_list = []
parking_list = []
for i in range(len(df3)):
    bedroom_list.append(re.search('\d+', df3['rooms'][i][0][0]).group())
    bathroom_list.append(re.search('\d+', df3['rooms'][i][1][0]).group())
    if(len(df3['rooms'][i][2])!= 0):
        parking_list.append(re.search('\d+', df3['rooms'][i][2][0]).group())
    else:
        parking_list.append('0')
    # df3['bathroom'][i] = re.search('\d+', df3['rooms'][i][1][0]).group()
    # df3['parking'][i] = re.search('\d+', df3['rooms'][i][2][0]).group()

In [319]:
df3['bedroom'] = bedroom_list
df3['bathroom'] = bathroom_list
df3['parking'] = parking_list

In [320]:
def lowercase(text):
    text = text.lower()
    return text
df3['cost_text'] = df3['cost_text'].apply(lowercase)

In [323]:
def amount(text):
    num = re.findall(r'\d*[,.]?\d+[,.]?\d+',text)
    if len(num) == 0:
        return None
    elif len(num) == 1:       
        weekly_rent = float(num[0].replace(',',''))
    else:
        if('high' in text and 'low' in text):
            high = float(re.findall(r'\$\d+,?.\d+.?\d+ pw high',text)[0].split(' ')[0][1:].replace(',',''))
            low = float(re.findall(r'\$\d+,?.\d+.?\d+ pw low',text)[0].split(' ')[0][1:].replace(',',''))
            weekly_rent = (high + low)/2
        elif('per week' in text or 'pw' in text):
            num_text = re.findall(r'\$\d+,?.\d+.?\d+ pw', text)
            if(len(num_text) == 1):
                weekly_rent = float(num_text[0].split(' ')[0][1:].replace(',',''))
            else:
                num_text = re.findall(r'\$\d*[,.]?\d+[,.]?\d+pw', text)
                if(len(num_text) == 1):
                    weekly_rent = float(num_text[0][1:-2].replace(',',''))
                elif len(num_text) == 0:
                    weekly_rent = float(re.findall(r'\d*[,.]?\d+[,.]?\d+ per week', '$450 per week ($1955 pcm)')[0].split(' ')[0])
                else:
                    print("1")
                    print(text)
        else:
            return None

       
    if('annually' in text):
        rent = weekly_rent/52
    elif('month' in text):
        rent = weekly_rent/4
    else:
        rent = weekly_rent
    return rent
        
        


In [324]:
df3['amount'] = df3['cost_text'].apply(amount)

In [325]:
df3['amount'].isnull().sum()

26

In [326]:
def postcode(text):
    postcode = re.findall(r'VIC \d+',text)
    if(len(postcode) == 0):
        return None
    else:
        return postcode[0].split(' ')[1]
   

In [327]:
df3['postcode'] = df3['name'].apply(postcode)

In [328]:
def bond(text):
    if(text is None):
        return None
    if('Bond' in text):
        amount = re.findall(r'Bond \$\d+',text)
        if (len(amount) == 1):
            amount = float(amount[0].split(' ')[1][1:])
            return amount
   
    else:
        return None


In [329]:
df3['bond'] = df3['external_information'].apply(bond)
df3

,link,name,cost_text,coordinates,rooms,external_information,desc,bedroom,bathroom,parking,amount,postcode,bond
0,https://www.domain.com.au/26-55-alexandra-st-s...,26/55 Alexandra St St Kilda East VIC 3183,$330 weekly,"[-37.8644943, 145.0029153]","[[1 Bed], [1 Bath], [1 Parking]]","Available from Thursday, 1st September 2022Bon...",Leasing Concierge,1,1,1,330.0,3183,1434.0
1,https://www.domain.com.au/1-27-young-street-gl...,1/27 Young Street Glen Iris VIC 3146,$695.00 per week,"[-37.860268, 145.0435189]","[[3 Beds], [2 Baths], [2 Parking]]",Date Available: Available Now,Maree Naylor,3,2,2,695.0,3146,NaN
2,https://www.domain.com.au/3405-288-spencer-str...,3405/288 Spencer Street Melbourne VIC 3000,$440,"[-37.813775, 144.9520948]","[[1 Bed], [1 Bath], []]","Available from Friday, 16th September 2022Bond...",Kim Te,1,1,0,440.0,3000,1912.0
3,https://www.domain.com.au/26-victoria-grove-ha...,26 Victoria Grove Hawthorn East VIC 3123,$995.00,"[-37.8220902, 145.0488853]","[[3 Beds], [2 Baths], [2 Parking]]",Date Available: Available NowBond $6485,Tom Macrae,3,2,2,995.0,3123,6485.0
4,https://www.domain.com.au/208-20-queen-street-...,208/20 Queen Street Blackburn VIC 3130,$360pw/$1.560pcm,"[-37.8192859, 145.1509058]","[[1 Bed], [1 Bath], [1 Parking]]",Date Available: Available NowBond $1560,Anthony Blight,1,1,1,360.0,3130,1560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,https://www.domain.com.au/7-atherton-way-werri...,7 Atherton Way Werribee VIC 3030,$410 pw,"[-37.9010132, 144.598294]","[[4 Beds], [2 Baths], [2 Parking]]",Date Available: Available NowBond $1782,Centrally located in the heart of the popular ...,4,2,2,410.0,3030,1782.0
2998,https://www.domain.com.au/27-gordon-street-too...,27 Gordon Street Toorak VIC 3142,$695.00 per week,"[-37.8437917, 145.0068204]","[[2 Beds], [1 Bath], [1 Parking]]",Date Available: Available NowBond $3020,This charming light filled two bedroom residen...,2,1,1,695.0,3142,3020.0
2999,https://www.domain.com.au/18-college-road-werr...,18 College Road Werribee VIC 3030,$390,"[-37.90651829999999, 144.6654842]","[[4 Beds], [2 Baths], [2 Parking]]",Date Available: Available NowBond $1695,"In keeping with its cottage charm, this proper...",4,2,2,390.0,3030,1695.0
3000,https://www.domain.com.au/b30-berth-martha-cov...,B30 BERTH MARTHA COVE MARINA Safety Beach VIC ...,$160.00,"[-38.3352183, 144.9942305]","[[1 Bed], [1 Bath], []]","Available from Friday, 25th November 2022Bond ...","BERTH 30 MARTHA COVE, SAFETY BEACH - BERTH ONL...",1,1,0,160.0,3936,200.0


In [330]:
df3.to_csv("../data/curated/cleaned.csv")